# Data Preprocessing

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [3]:
from google.colab import files
uploaded = files.upload()


Saving ner_dataset.csv to ner_dataset.csv


In [4]:
df = pd.read_csv('ner_dataset.csv', encoding= 'unicode_escape')
df.fillna(method = 'ffill', inplace = True)
df.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [5]:
df = df.drop(['POS'], axis=1)
df = df.groupby('Sentence #').agg(list)
df = df.reset_index(drop=True)
df.head()

,Word,Tag
0,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,"[Iranian, officials, say, they, expect, to, ge...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
2,"[Helicopter, gunships, Saturday, pounded, mili...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O..."
3,"[They, left, after, a, tense, hour-long, stand...","[O, O, O, O, O, O, O, O, O, O, O]"
4,"[U.N., relief, coordinator, Jan, Egeland, said...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo..."


In [83]:
labels = list(set([label for sent in df['Tag'] for label in sent]))
label2idx = {label: idx for idx, label in enumerate(labels)}

In [85]:
print(label2idx)

{'B-eve': 0, 'I-geo': 1, 'B-art': 2, 'B-nat': 3, 'B-org': 4, 'I-tim': 5, 'O': 6, 'I-art': 7, 'B-per': 8, 'I-eve': 9, 'B-geo': 10, 'B-gpe': 11, 'I-org': 12, 'I-gpe': 13, 'B-tim': 14, 'I-per': 15, 'I-nat': 16}


In [89]:
from future.utils import iteritems

idx2label = {v: k for k, v in iteritems(label2idx)}

In [75]:
tags = list(label2idx.keys())

In [76]:
print(tags)

['B-eve', 'I-geo', 'B-art', 'B-nat', 'B-org', 'I-tim', 'O', 'I-art', 'B-per', 'I-eve', 'B-geo', 'B-gpe', 'I-org', 'I-gpe', 'B-tim', 'I-per', 'I-nat']


In [14]:
words = list(set([word for sent in df['Word'] for word in sent]))
words.append('PAD')  # Padding token
word2idx = {word: idx for idx, word in enumerate(words)}


In [15]:
print(word2idx)

{'2,698': 0, 'Ben-Eliezer': 1, 'disciplines': 2, 'Falcons': 3, 'instructed': 4, 'lodged': 5, 'entitlement': 6, 'apologized': 7, 'Elham': 8, 'drunk': 9, 'aerial': 10, 'Seyni': 11, '178': 12, 'non-peaceful': 13, 'Nikiforov': 14, 'pleases': 15, 'pardon': 16, 'evangelization': 17, '1,254': 18, 'delinquency': 19, 'sabotaged': 20, 'manipulation': 21, 'justification': 22, 'offender': 23, 'Rodney': 24, 'behave': 25, 'Beitenu': 26, 'Twelve': 27, 'Scorpion': 28, 'Hourmadji': 29, 'technician': 30, 'Soyuz': 31, 'Schrock': 32, 'Mindaoudou': 33, 'anti-NATO': 34, '1.8': 35, 'respects': 36, 'Shimbun': 37, 'unbeaten': 38, 'beset': 39, 'Outgoing': 40, 'Hurricane': 41, 'airing': 42, 'Baalbek': 43, 'win': 44, 'Rights': 45, 'hearty': 46, 'Johnston': 47, 'decree': 48, 'marking': 49, 'maneuvers': 50, 'Siphiwe': 51, 'mansions': 52, 'Bong-jo': 53, 'boosts': 54, 'zookeepers': 55, '163': 56, 'spoke': 57, 'Religious': 58, 'Rodina': 59, 'obstructed': 60, 'preliminary': 61, 'barreled': 62, 'landmine': 63, 'photogra

In [16]:
df['word_idx'] = df.Word.apply(lambda x:[word2idx[word] for word in x])

In [8]:
df['Tag_idx'] = df['Tag'].apply(lambda x: [label2idx[label] for label in x])


In [17]:
df.head()

,Word,Tag,Tag_idx,word_idx
0,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[6, 6, 6, 6, 6, 6, 10, 6, 6, 6, 6, 6, 10, 6, 6...","[32858, 3836, 25564, 18110, 20700, 17852, 2463..."
1,"[Iranian, officials, say, they, expect, to, ge...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[11, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...","[15824, 21077, 17812, 30023, 25663, 13326, 504..."
2,"[Helicopter, gunships, Saturday, pounded, mili...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[6, 6, 14, 6, 6, 6, 6, 6, 10, 6, 6, 6, 6, 6, 4...","[19166, 32934, 34906, 12013, 29071, 26619, 166..."
3,"[They, left, after, a, tense, hour-long, stand...","[O, O, O, O, O, O, O, O, O, O, O]","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]","[28165, 13040, 34154, 27114, 23416, 5442, 224,..."
4,"[U.N., relief, coordinator, Jan, Egeland, said...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[10, 6, 6, 8, 15, 6, 14, 6, 10, 6, 11, 6, 11, ...","[20018, 10429, 34388, 27188, 2524, 2737, 21041..."


In [21]:
n_token = len(word2idx)
n_tag = len(label2idx)

In [22]:
print(n_token, n_tag)

35178 17


In [23]:
token = df['word_idx'].tolist()

In [24]:
maxlen = max([len(i) for i in token])

In [25]:
print(maxlen)

104


In [27]:
tag = df['Tag_idx'].tolist()

In [28]:
X = pad_sequences(token, maxlen=maxlen, dtype='int32',padding='post',value=n_token-1)

In [31]:
print(X[10])

[30388 26495 26775 27114  4175  3836 21077  1111  4749 21104 26775 30067
 34133 34751 23826 23207 18551 16697 33609 23397 26775 19264  9359  5404
  9298  2737 34751  7552  7002 35098  1548 16697 17933 21665 31668 33404
  9168  4749 19492 25200 35177 35177 35177 35177 35177 35177 35177 35177
 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177
 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177
 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177
 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177 35177
 35177 35177 35177 35177 35177 35177 35177 35177]


In [32]:
y = pad_sequences(tag, maxlen=maxlen, dtype='int32', padding='post', value= label2idx["O"])

In [34]:
print(y[0])

[ 6  6  6  6  6  6 10  6  6  6  6  6 10  6  6  6  6  6 11  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6]


In [35]:
y = [to_categorical(i, num_classes=n_tag) for i in y]

In [36]:
print(y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [37]:
X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=2000)

In [39]:
print(X_train.shape, X_test.shape)

(38367, 104) (9592, 104)


In [44]:
for tokens, tags in zip(X_train[0], y_train[0]):
    print('%s\t%s' % (tokens, tags))

11855	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
23707	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
21530	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
34751	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
5907	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
34154	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
15512	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
15348	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
6019	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
22820	[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
10819	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
4945	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
27886	[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
31130	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
30537	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
3069	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
25111	[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1

In [61]:
model = Sequential()
model.add(Embedding(35179, 32, input_length=104))
model.add(Dropout(0.5))
model.add(LSTM(100, return_sequences=True))
model.add(TimeDistributed(Dense(len(label2idx), activation='softmax')))

In [62]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 104, 128)          4502784   
                                                                 
 dropout (Dropout)           (None, 104, 128)          0         
                                                                 
 lstm (LSTM)                 (None, 104, 100)          91600     
                                                                 
 time_distributed (TimeDist  (None, 104, 17)           1717      
 ributed)                                                        
                                                                 
Total params: 4596101 (17.53 MB)
Trainable params: 4596101 (17.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
history = model.fit(X_train, np.array(y_train), batch_size=32, epochs=7, validation_split=0.1)

Epoch 1/7
1080/1080 [==============================] - 66s 60ms/step - loss: 0.1564 - accuracy: 0.9710 - val_loss: 0.0718 - val_accuracy: 0.9792
Epoch 2/7
1080/1080 [==============================] - 64s 60ms/step - loss: 0.0587 - accuracy: 0.9834 - val_loss: 0.0435 - val_accuracy: 0.9888
Epoch 3/7
1080/1080 [==============================] - 65s 60ms/step - loss: 0.0368 - accuracy: 0.9900 - val_loss: 0.0324 - val_accuracy: 0.9910
Epoch 4/7
1080/1080 [==============================] - 64s 59ms/step - loss: 0.0287 - accuracy: 0.9918 - val_loss: 0.0289 - val_accuracy: 0.9913
Epoch 5/7
1080/1080 [==============================] - 66s 61ms/step - loss: 0.0250 - accuracy: 0.9926 - val_loss: 0.0274 - val_accuracy: 0.9916
Epoch 6/7
1080/1080 [==============================] - 64s 59ms/step - loss: 0.0230 - accuracy: 0.9930 - val_loss: 0.0268 - val_accuracy: 0.9919
Epoch 7/7
1080/1080 [==============================] - 64s 60ms/step - loss: 0.0215 - accuracy: 0.9933 - val_loss: 0.0269 - val_ac

In [78]:
y_pred = model.predict(np.array([X_test[32]]))

1/1 [==============================] - 0s 27ms/step


In [79]:
y_pred = np.argmax(y_pred,axis =-1)
get=np.argmax(y_test[32],axis=-1)

In [80]:
print(get)

[ 6  6  6  6  6  6  6 10  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
  6  6  6  6  6  6  6  6]


In [82]:
get[0]

6

In [91]:
print("{:14}: ({:5}): {}".format("Word", "True", "Pred"))
for idx, (w, pred) in enumerate(zip(X_test[32], y_pred[0])):
    print("{:14}: ({:5}): {}".format(words[w], idx2label[get[idx]], tags[pred]))

Word          : (True ): Pred
The           : (O    ): O
agreement     : (O    ): O
triggered     : (O    ): O
a             : (O    ): O
political     : (O    ): O
crisis        : (O    ): O
in            : (O    ): O
Ukraine       : (B-geo): B-geo
,             : (O    ): O
and           : (O    ): O
parliament    : (O    ): O
voted         : (O    ): O
to            : (O    ): O
dismiss       : (O    ): O
the           : (O    ): O
government    : (O    ): O
.             : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O    ): O
Ojeda         : (O   

In [92]:
Y_pred = model.predict(np.array(X_test))

300/300 [==============================] - 8s 25ms/step


In [93]:
Y_pred.shape

(9592, 104, 17)

In [94]:
from sklearn.metrics import classification_report

In [95]:
print(classification_report(np.argmax(y_test, 2).ravel(), np.argmax(Y_pred, axis=2).ravel(),labels=list(idx2label.keys()), target_names=list(idx2label.values())))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-eve       0.93      0.22      0.36        59
       I-geo       0.82      0.71      0.76      1469
       B-art       0.00      0.00      0.00        73
       B-nat       0.00      0.00      0.00        46
       B-org       0.65      0.65      0.65      3986
       I-tim       0.90      0.61      0.72      1249
           O       1.00      1.00      1.00    965567
       I-art       0.00      0.00      0.00        60
       B-per       0.85      0.78      0.81      3469
       I-eve       0.33      0.02      0.04        45
       B-geo       0.84      0.84      0.84      7597
       B-gpe       0.94      0.94      0.94      3174
       I-org       0.71      0.76      0.73      3254
       I-gpe       1.00      0.27      0.42        45
       B-tim       0.93      0.80      0.86      3913
       I-per       0.85      0.86      0.86      3550
       I-nat       0.00      0.00      0.00        12

    accuracy              

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Les résultats de l'évaluation du modèle de classification pour l'étiquetage des entités nommées montrent une performance globalement solide avec une précision élevée et une excellente précision globale (99%). Cependant, une analyse plus détaillée révèle des disparités significatives entre les différentes classes.

Les classes telles que 'B-geo', 'B-gpe', 'B-tim', 'I-per', présentent de bons scores de précision, de rappel et de F1-score, suggérant que le modèle réussit bien à identifier ces entités nommées.

En revanche, certaines classes comme 'B-eve', 'B-art', 'B-nat', 'I-art', 'I-eve', 'I-nat' montrent des performances très faibles en termes de rappel et de F1-score. Cela indique que le modèle a du mal à identifier ces types spécifiques d'entités nommées. Une amélioration de la capacité du modèle à reconnaître ces catégories moins fréquentes ou plus complexes pourrait être nécessaire.

En conclusion, bien que le modèle obtienne une précision globale élevée, il est important de prendre en compte les performances individuelles par classe.





